def recomendacion_usuario( id de usuario ): Ingresando el id de un usuario, deberíamos recibir una lista con 5 juegos recomendados para dicho usuario.

In [4]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats 

In [15]:
review = pd.read_parquet("clean_review")
items = pd.read_parquet("clean_items")
steam = pd.read_parquet("clean_steam_games")

In [16]:
items_y_steam = pd.merge(items, steam, on= "item_id")

In [17]:
review_y_steam = pd.merge(review, steam, on="item_id")

In [18]:
items_y_steam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4294257 entries, 0 to 4294256
Data columns (total 32 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   user_id                    object 
 1   items_count                int64  
 2   item_id                    float64
 3   item_name                  object 
 4   playtime_forever           float64
 5   playtime_2weeks            float64
 6   app_name                   object 
 7   price                      float64
 8   developer                  object 
 9   Accounting                 int64  
 10  Action                     int64  
 11  Adventure                  int64  
 12  Animation &amp; Modeling   int64  
 13  Audio Production           int64  
 14  Casual                     int64  
 15  Design &amp; Illustration  int64  
 16  Early Access               int64  
 17  Education                  int64  
 18  Free to Play               int64  
 19  Indie                      int64  
 20  Ma

In [19]:
items_y_steam.isna().sum().sort_values(ascending= False)

developer                    106159
Año_lanzamiento               78438
price                          2445
Web Publishing                    0
Video Production                  0
Utilities                         0
Strategy                          0
Sports                            0
Education                         0
Software Training                 0
Simulation                        0
Racing                            0
RPG                               0
Photo Editing                     0
Massively Multiplayer             0
Indie                             0
Free to Play                      0
user_id                           0
items_count                       0
Design &amp; Illustration         0
Casual                            0
Audio Production                  0
Animation &amp; Modeling          0
Adventure                         0
Action                            0
Accounting                        0
app_name                          0
playtime_2weeks             

In [22]:
items_y_steam.drop_duplicates(inplace= True)

In [23]:
items_y_steam.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4244831 entries, 0 to 4294256
Data columns (total 32 columns):
 #   Column                     Dtype  
---  ------                     -----  
 0   user_id                    object 
 1   items_count                int64  
 2   item_id                    float64
 3   item_name                  object 
 4   playtime_forever           float64
 5   playtime_2weeks            float64
 6   app_name                   object 
 7   price                      float64
 8   developer                  object 
 9   Accounting                 int64  
 10  Action                     int64  
 11  Adventure                  int64  
 12  Animation &amp; Modeling   int64  
 13  Audio Production           int64  
 14  Casual                     int64  
 15  Design &amp; Illustration  int64  
 16  Early Access               int64  
 17  Education                  int64  
 18  Free to Play               int64  
 19  Indie                      int64  
 20  Massive

Como el objetivo de esta etapa es armar eun modelo de recomendacion de juego por usuario, eliminamos todas las filas que no tengan usuario y juego
